In [1]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import plotnine as p9
from functools import partial
import itertools

In [2]:
# create the grid of pairs (consumption today, tomorrow)
amounts_consumption = np.linspace(0, 1000, 100).round(0).astype(int)

def expand_grid(data_dict):
    """
    SOURCE: https://pandas.pydata.org/docs/user_guide/cookbook.html
    """
    rows = itertools.product(*data_dict.values())
    return pd.DataFrame.from_records(rows, columns=data_dict.keys())

outcomes_wellbeing = expand_grid({'consumption_today': amounts_consumption, 'consumption_tomorrow': amounts_consumption})

In [ ]:
transform_wellbeing_power_form(10, 0.5)

In [4]:
# classical discount factor equals 1 / risk_free_gross_rate; 
# suppose 5% risk free net rate
SUBJECTIVE_DISCOUNT_FACTOR = 1 / 1.05
STRENGTH_OF_DIMINISHING = 0.5

wellbeing_curve = partial(transform_wellbeing_power_form, strength_of_diminishing=STRENGTH_OF_DIMINISHING)

lifetime_wellbeing_curve = partial(
    transform_lifetime_wellbeing, 
    wellbeing_func=wellbeing_curve, 
    subjective_discount_factor=SUBJECTIVE_DISCOUNT_FACTOR
    )

outcomes_wellbeing['lifetime_wellbeing'] = lifetime_wellbeing_curve(
    consumption_today=outcomes_wellbeing['consumption_today'], 
    consumption_expected_tomorrow=outcomes_wellbeing['consumption_tomorrow']
    )

In [ ]:
# 3d area plotly requires a particular data structure.
# examples reference: https://plotly.com/python/3d-surface-plots/
todays_wellbeing = outcomes_wellbeing.pivot(index='consumption_today', columns='consumption_tomorrow')['lifetime_wellbeing']

# choose x and y axis series to clarify the user's view.
# decision interacts with the 3d visual's angle presented
y=todays_wellbeing.index.values
x=todays_wellbeing.columns.values
z=todays_wellbeing.values.round(1)
titles_axes = dict(
    yaxis_title_text="Consumption Today",
    xaxis_title_text='Consumption <br>Expected Tomorrow', 
    zaxis_title_text="Lifetime Wellbeing"
    )

fig = go.Figure(data=[go.Surface(z=z, x=x, y=y)])

fig.update_traces(
    contours_z=dict(show=True, usecolormap=True, highlightcolor="limegreen", project_z=True),
    hovertemplate=(
        "Consumption Today: %{y}<br>" + 
        "Consumption Expected Tomorrow: %{x}<br>" + 
        "Lifetime Wellbeing: %{z}<br>" + 
        "<extra></extra>" # remove trace name
        )
    )

# better view in the web browser when autosize allowed
fig.update_layout(
    title=dict(text='Lifetime Wellbeing Varies with Consumption'), 
    scene_camera_eye=dict(x=2.5, y=0.88, z=0.5),
    )

# 3d plots axis titles have special access method, vs 2d
# reference: https://community.plotly.com/t/how-to-update-x-y-and-z-axes-titles-for-3d-surface-plots-within-a-subplot/45495
fig.update_scenes(**titles_axes)

fig.show()

# fig.write_html("./docs/lifetime_wellbeing_plot.html", include_plotlyjs='cdn')

In [10]:
wellbeing_dwdc_curve = partial(transform_wellbeing_power_form_dwdc, strength_of_diminishing=STRENGTH_OF_DIMINISHING)

price_curve = partial(infer_price, wellbeing_dwdc=wellbeing_dwdc_curve, subjective_discount_factor=SUBJECTIVE_DISCOUNT_FACTOR)

def present_price_wellbeing(consumption_today, consumption_tomorrow, payoff_asset):

    lifetime_wellbeing = lifetime_wellbeing_curve(
        consumption_today=consumption_today, 
        consumption_expected_tomorrow=consumption_tomorrow
        )
    
    price = price_curve(
        consumption_today=consumption_today, 
        consumption_tomorrow=consumption_tomorrow, 
        payoff_asset=payoff_asset
        )

    print(f"Lifetime wellbeing: {lifetime_wellbeing}, price: {price}")

In [ ]:
# hypothesis: when asset payoff rises, then price rises
present_price_wellbeing(424, 424, 100)
present_price_wellbeing(424, 424, 200)

In [ ]:
# hypothesis: if consumption_today is high, consumption_expected_tomorrow is low, then price is high
present_price_wellbeing(424, 424, 100)
present_price_wellbeing(424, 324, 100)
present_price_wellbeing(424, 224, 100)

In [ ]:
# hypothesis: if consumption_today is low, consumption_expected_tomorrow is high, then price is low
present_price_wellbeing(424, 424, 100)
present_price_wellbeing(324, 424, 100)
present_price_wellbeing(224, 424, 100)

# Appendix

In [ ]:
break

In [ ]:
# intuit how wellbeing curve varies with gamma
tunes_wellbeing = []
for g in [0, 0.1, 0.25, 0.5, 0.75, 0.9]:
    outcomes_wellbeing = transform_wellbeing(amounts_consumption, g)
    outcomes_wellbeing = pd.DataFrame({
        'consumption': amounts_consumption,
        'wellbeing': outcomes_wellbeing,
        'gamma': g
    })
    tunes_wellbeing.append(outcomes_wellbeing)

tunes_wellbeing = pd.concat(tunes_wellbeing, axis=0, ignore_index=True)

In [ ]:
(
    p9.ggplot(tunes_wellbeing) + 
    p9.theme_bw() + 
    p9.geom_line(p9.aes('consumption', 'wellbeing', color='gamma', group='gamma')) + 
    p9.theme(figure_size=(8,3))
)

In [168]:
# example shows the special data structure expected by plotly
# https://plotly.com/python/3d-scatter-plots/

# Read data from a csv
# z_data = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/api_docs/mt_bruno_elevation.csv')
# z = z_data.values
# sh_0, sh_1 = z.shape
# x, y = np.linspace(0, 1, sh_0), np.linspace(0, 1, sh_1)
# fig = go.Figure(data=[go.Surface(z=z, x=x, y=y)])
# fig.update_layout(title=dict(text='Mt Bruno Elevation'), autosize=False,
#                   width=500, height=500,
#                   margin=dict(l=65, r=50, b=65, t=90))
# fig.show()

In [ ]:
# test: can i embed a plotly chart via html?
fig = go.Figure(data=[go.Scatter(x=[1, 2, 3], y=[1, 4, 9])])

# successful alternative: host via GitHub Pages
# https://community.plotly.com/t/how-to-display-plotly-graph-on-github-pages/44398/3
# steps:
    # enable GitHub Pages in my repo
    # write plot to html file
# fig.write_html("./docs/test_plot.html", include_plotlyjs='cdn')
# https://abrahamalex13.github.io/learning-to-price-assets/test_plot

# did not work: pasting below in Substack html editor
# <div class="plotly-plot"> {{ html_string }} </div>
# where html_string -> 
    # html_string = fig.to_html(full_html=False, include_plotlyjs='cdn')
    # print(html_string)